# 1. Descarga de archivos grb2 provenientes de los trabajos del centro de modelamiento ambiental del NOAA.

En este punto se usará el paquete wget, el cual es un paquete desarrollado en la función wget usada linux. Para esto se necesita:
1.   Importar funcion descargasarchivos_multi1 y descargas_era5copernicus.
2.   Ejecutar cada uno de los codigos para el rango temporal de 2009 a 2018. En este aspecto solo se consideran los datos provenientes del NOAA, los cuales fueron usados en la tesis.

In [18]:
import sys
sys.path.insert(0, 'Script')

import numpy as np
import itertools
from S001_download import descargasarchivos_multi1
from S002_swanfiles import conversiongrib2toCSVNOAA

# Descargar archivos provenientes del NOAA
path_folder_NOAA = "Data/grib_NOAA"
min_year         = 2009
max_year         = 2018
path_folder_swn  = "Output/swn"
path_folder_csv  = "Output/csv"
for var in ['dp','hs','tp']:
    descargasarchivos_multi1(path_folder_NOAA, var, min_year, max_year)

termino iteración de variable: dp
termino iteración de variable: hs
termino iteración de variable: tp


In [19]:
import math

def round_up(n, decimals=0):
    multiplier = 10 ** decimals
    return math.ceil(n * multiplier) / multiplier

def round_down(n, decimals=0):
    multiplier = 10 ** decimals
    return math.floor(n * multiplier) / multiplier

def get_values_from_nodos(i, df, nodos):
    x1, y1, x2, y2 = nodos[i]
    subdf = df.loc[(df['longitude']==x1)&(df["latitude"]==y1)]
    hs, tp, dp = subdf[["swh","perpw","dirpw"]].values[0]
    return x1, y1, x2, y2, hs, tp, dp

def model_parameters(key, sub_key, dictionary):
    if sub_key == "grid":
        xpc, ypc, alpc, xlenc, ylenc, mxc, myc, mdc, flow, fhigh = dictionary[key][sub_key]
        msc = round_up(np.log10(fhigh/flow)/np.log10(1+0.1))
        return xpc, ypc, alpc, xlenc, ylenc, mxc, myc, mdc, flow, fhigh, msc
    elif sub_key == "bathymetry":
        xpinp, ypinp, alpinp, mxinp, myinp, arc_seconds = dictionary[key][sub_key]
        dxyinp = arc_seconds*(1/3600)
        return xpinp, ypinp, alpinp, mxinp, myinp, dxyinp, dxyinp
    elif sub_key == "names_bath":
        fac, name, idla, nhedf = dictionary[key][sub_key]
        return fac, name, idla, nhedf
    elif sub_key == "numeric":
        dabs, drel, curvat, npnts, mxitst, alfa = dictionary[key][sub_key]
        return dabs, drel, curvat, npnts, mxitst, alfa
    elif sub_key == "subgrid":
        xpn, ypn, alpn, xlen, ylen, mxn, myn = dictionary[key][sub_key]
        return xpn, ypn, alpn, xlen, ylen, mxn, myn
    elif sub_key == "group":
        ix1, ix2, iy1, iy2 = dictionary[key][sub_key]
        return ix1, ix2, iy1, iy2
    elif sub_key == "ray":
        xp1, yp1, xq1, yq1, nint, xp, yp, xq, yq = dictionary[key][sub_key]
        return xp1, yp1, xq1, yq1, nint, xp, yp, xq, yq
    elif sub_key == "output":
        ndec, len = dictionary[key][sub_key]
        return ndec, len


def EscrituraSwanFile(df, path_out, model, grid, nodos, parameters):
    for n, time in enumerate(df.time):
        if n<20:
            subdf = df[df['time']==time]
            fecha = time.strftime("%Y%m%dT%H")
            name_file = path_out+"/"+fecha+"_"+model+".swn"
            file = open(name_file, "w")
            file.write(
                f'$******************************  HEADING  *************************************\n'
                f'PROJ \'Memoria\' \'M\' \n'
                f'$ OBTENCION ESPECTROS ANIDACION PARA MODELO: {model}, HORA: {time} \n'
                f'$**************************   MODEL STARTUP  **********************************\n'
                f'SET NAUTical\nCOORDinates SPHErical CCM\n'
                f'$****************************  MODEL INPUT  ***********************************\n'
                f'CGRID REGular %.2f %.2f %.2f %.2f %.2f %i %i CIRCLE %i %.2f %.2f %i' %(model_parameters(model, "grid", grid)) +'\n'
                f'INPgrid BOTtom REGular %.5f %.5f %.2f %d %d %.7f %.7f' %(model_parameters(model, "bathymetry", grid)) +'\n'
                f'READinp BOTtom %.1f \'%s\' %i %i FREE' %(model_parameters(model, "names_bath", grid)) +'\n'
                f'$*****************    INPUT BOUNDARY SPECTRAL CONDITIONS   ********************\n'
                )
            for nodo_key in nodos.keys():
                file.write(
                    f'BOUNdspec SEGMent XY %.2f %.2f %.2f %.2f CONstant PAR %.2f %.2f %.2f' %(get_values_from_nodos(nodo_key, subdf, nodos)) +'\n'
                    )
            file.write(
                f'$****************************     PHYSICS    **********************************\n'
                f'BREAking CONSTANT 1.0 0.42\n'
                f'OFF QUADRUPL\nOFF WCAPPING\n'
                f'$***************************      NUMERICS   **********************************\n'
                f'NUM STOPC dabs=%.3f drel=%.3f curvat=%.3f npnts=%.3f STAT mxitst=%i alfa=%.3f' %(model_parameters(model, "numeric", grid)) +'\n'
                f'$***************************** OUTPUT REQUESTS  *******************************\n'
                )
            if model=="P1":
                file.write(
                    f'NGRid \'P2\' %.2f %.2f %.2f %.2f %.2f %i %i '%(model_parameters(model, "subgrid", grid)) +'\n'
                    f'NESTout \'P2\' \'Output/dat/{fecha}_P2.dat\' \n'
                    )
            elif model=="P2":
                file.write(
                    f'NGRid \'P3\' %.2f %.2f %.2f %.2f %.2f %i %i '%(model_parameters(model, "subgrid", grid)) +'\n'
                    f'NESTout \'P3\' \'Output/dat/{fecha}_P3.dat\' \n'
                    )
            file.write(
                f'$\n'
                f'GROUP \'OUTPUT\' SUBGRID %i %i %i %i' %(model_parameters(model, "group", grid)) +'\n'
                f'OUTPut OPTIons BLOCK %i %i' %(model_parameters(model, "output", grid)) +'\n'
                f'BLOCK \'OUTPUT\' NOHEADER \'Output/block-mat/{fecha}_{model}.mat\' LAY-OUT 1 '+parameters+' \n'
                f'$\n'
                # f'POINts \'Points\' FILE \'Points.pnt\' \n'
                # f'TABLe \'Points\' NOHEADER \'Output/point/{fecha}_{model}.txt\' '+parameters+' \n'
                # f'$\n'
                f'RAY \'RAY\' %.2f %.2f %.2f %.2f %i %.2f %.2f %.2f %.2f' %(model_parameters(model, "ray", grid)) +'\n'
                )
            for depth in np.arange(50, 200 + 50, 50):
                name_veril = "V0%i" %(depth) if (depth < 100) else "V%i" %(depth)
                file.write(
                    f'ISOline \'%s\' \'RAY\' DEPth %i' %(name_veril, depth) +'\n'
                    f'TABLe \'%s\' NOHEADER \'Output/isoline/{fecha}_{model}_%s.txt\' '%(name_veril, name_veril) +parameters+' \n'
                )
            file.write(
                f'$\n'
                f'TEST 1,0\n'
                f'COMPUTE\n'
                f'STOP')
            file.close()
    return

area = [-32.5, -72.5, -33.5, -71.5] #WGS84
nodos = {
    "nodo1": [-71.50, -32.50, -72.00, -32.50],
    "nodo2": [-72.00, -32.50, -72.50, -32.50],
    "nodo3": [-72.50, -32.50, -72.50, -33.00],
    "nodo4": [-72.50, -33.00, -72.50, -33.50],
    "nodo5": [-72.50, -33.50, -72.00, -33.50],
    "nodo6": [-72.00, -33.50, -71.40, -33.50]
    }
grid = {
    "P1": {
        "grid"      : [-72.50, -33.50, 0.00, 1.10, 1.00, 66, 60, 180, 0.02, 1],
        "bathymetry": [-72.50625, -33.50625, 0.0, 291, 241, 15], 
        "names_bath": [-1, "bat_final3.bot", 3, 0],
        "numeric"   : [0.005, 0.01, 0.005, 98.00, 50, 0.01],
        "subgrid"   : [-72.00, -33.25, 0.0, 0.60, 0.50, 72, 60],
        "group"     : [0, 66, 0, 60],
        "ray"       : [-72.50, -33.50, -71.40, -33.50, 61, -72.50, -32.50, -71.40, -32.50],
        "output"    : [9, 66],
        },
    "P2":{
        # "grid"      : [-72.50, -33.50, 0.00, 1.10, 1.00, 66, 60, 180, 0.02, 1],
        # "bathymetry": [-72.50625, -33.50625, 0.0, 291, 241, 15], 
        # "names_bath": [-1, "bat_final3.bot", 3, 0],
        # "numeric"   : [0.005, 0.01, 0.005, 98.00, 50, 0.01],
        # "subgrid"   : [-72.00, -33.25, 0.0, 0.60, 0.50, 72, 60],
        # "group"     : [0, 66, 0, 60],
        # "ray"       : [-72.50, -33.50, -71.40, -33.50, 61, -72.50, -32.50, -71.40, -32.50]
    },
    "P3":{
        # "grid"      : [-72.50, -33.50, 0.00, 1.10, 1.00, 66, 60, 180, 0.02, 1],
        # "bathymetry": [-72.50625, -33.50625, 0.0, 291, 241, 15], 
        # "names_bath": [-1, "bat_final3.bot", 3, 0],
        # "numeric"   : [0.005, 0.01, 0.005, 98.00, 50, 0.01],
        # "subgrid"   : [-72.00, -33.25, 0.0, 0.60, 0.50, 72, 60],
        # "group"     : [0, 66, 0, 60],
        # "ray"       : [-72.50, -33.50, -71.40, -33.50, 61, -72.50, -32.50, -71.40, -32.50]
    }
}
parameters='XP YP DEPTH DIR HSIGN TPS &\nTM02 TMM10 HSWELL WLEN STEEPNESS BFI QP TRANSP FORCE TDIR'
df = conversiongrib2toCSVNOAA(path_folder_NOAA, path_folder_csv, area, min_year, max_year)

EscrituraSwanFile(df, path_folder_swn, "P1", grid, nodos, parameters)

In [3]:
# """# Descargar archivos provenientes de Copernicus
# # path_folder_copernicus = "Data/grib_Copernicus"
# # years  = np.arange(2011, 2020 + 1, 1, dtype=int).tolist()
# # variables              = ['mean_wave_direction', 'model_bathymetry', 'peak_wave_period', 'significant_height_of_total_swell',]
# # area                   = [-35, -76, -39, -72,]

# # for min_year, max_year in zip(years[::2], years[1::2]):
# #     descargas_era5copernicus(path_folder_copernicus, min_year, max_year, variables, area)"""

In [1]:
import xarray as xr
import pandas as pd
import numpy as np

ds = xr.open_mfdataset("Output/block-nc/*.nc", concat_dim="time", combine="nested")
ds = ds.assign_coords(time=pd.to_datetime(np.array(ds.time.values, dtype=np.int64), format="%Y%m%d%H%M%S"))
ds

NameError: name 'np' is not defined